In [2]:
import collections
import math
import os
import shutil
import pandas as pd
import torch
import torchvision
from torch import nn
from d2l import torch as d2l
import time

import os
import shutil
import collections
import math
import time
import torchvision
import torch
import torch.nn as nn
import torch.utils.data

import datetime



In [3]:
import torch
import torchvision
import torchvision.transforms as transforms
import torch.nn as nn
import torch.optim
import time
import matplotlib.pyplot as plt

from torch.utils.data import DataLoader, random_split


from d2l import torch as d2l  # 确保已经安装了 d2l 包


if torch.cuda.is_available():
    devices = [torch.device('cuda')]
    print(f"Training on {torch.cuda.get_device_name(0)}")
else:
    devices = [torch.device('cpu')]
    print("Training on CPU")


# 数据预处理
transform_train = transforms.Compose([
    transforms.Resize(40),
    transforms.RandomResizedCrop(32, scale=(0.64, 1.0), ratio=(1.0, 1.0)),
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),
    transforms.Normalize([0.4914, 0.4822, 0.4465], [0.2023, 0.1994, 0.2010])
])

transform_test = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize([0.4914, 0.4822, 0.4465], [0.2023, 0.1994, 0.2010])
])
batch_size = 128

# 加载 CIFAR-10 数据集
test_ds = torchvision.datasets.CIFAR10(root='../data', train=False, download=True, transform=transform_test)

# 加载 CIFAR-10 训练集
full_train_ds = torchvision.datasets.CIFAR10(root='../data', train=True, download=True, transform=transform_train)

# 分割训练集和验证集
valid_ratio = 0.1  # 比如，使用 10% 的数据作为验证集
num_train = len(full_train_ds)
num_valid = int(num_train * valid_ratio)
train_ds, valid_ds = random_split(full_train_ds, [num_train - num_valid, num_valid])

# 创建 DataLoader
train_iter = DataLoader(train_ds, batch_size=batch_size, shuffle=True, num_workers=2)
valid_iter = DataLoader(valid_ds, batch_size=batch_size, shuffle=False, num_workers=2)


# 定义数据加载器
train_iter = torch.utils.data.DataLoader(train_ds, batch_size=batch_size, shuffle=True, num_workers=2)
test_iter = torch.utils.data.DataLoader(test_ds, batch_size=batch_size, shuffle=False, num_workers=2)
valid_ratio = 0.1
# 其余定义模型和训练的代码保持不变...
valid_iter = torch.utils.data.DataLoader(valid_ds, batch_size, shuffle=False,
                                         drop_last=True)

test_iter = torch.utils.data.DataLoader(test_ds, batch_size, shuffle=False,
                                        drop_last=False)
def get_net():
    num_classes = 10
    net = d2l.resnet18(num_classes, 3)
    return net

loss = nn.CrossEntropyLoss(reduction="none")

def train(net, train_iter, valid_iter, num_epochs, lr, wd, devices, lr_period,
          lr_decay):
    trainer = torch.optim.SGD(net.parameters(), lr=lr, momentum=0.9,
                              weight_decay=wd)
    scheduler = torch.optim.lr_scheduler.StepLR(trainer, lr_period, lr_decay)
    num_batches, timer = len(train_iter), d2l.Timer()
    legend = ['train loss', 'train acc']
    if valid_iter is not None:
        legend.append('valid acc')
    # animator = d2l.Animator(xlabel='epoch', xlim=[1, num_epochs],
    #                         legend=legend)
    net = nn.DataParallel(net, device_ids=devices).to(devices[0])
    train_loss_list, train_acc_list, valid_acc_list = [], [], []
    for epoch in range(num_epochs):
        net.train()
        metric = d2l.Accumulator(3)
        start = time.time()
        for i, (features, labels) in enumerate(train_iter):
            timer.start()
            l, acc = d2l.train_batch_ch13(net, features, labels.to(devices[0]), loss, trainer, devices)
            metric.add(l, acc, labels.shape[0])
            timer.stop()
            # if (i + 1) % (num_batches // 5) == 0 or i == num_batches - 1:
                # animator.add(epoch + (i + 1) / num_batches,
                #              (metric[0] / metric[2], metric[1] / metric[2],
                #               None))
        train_loss, train_acc = metric[0] / metric[2], metric[1] / metric[2]                          
        epoch_time = time.time() - start
        # print(f'Epoch {epoch + 1}, Loss {metric[0] / metric[2]:.3f}, Train acc {metric[1] / metric[2]:.3f}, Time {timer.sum():.1f} sec')
        if valid_iter is not None:
            valid_acc = d2l.evaluate_accuracy_gpu(net, valid_iter)
            # animator.add(epoch + 1, (None, None, valid_acc))
        print(f"{epoch+1}/{num_epochs} - {epoch_time:.0f}s - loss: {train_loss:.4f} - accuracy: {train_acc:.4f} - val_accuracy: {valid_acc:.4f} - {epoch_time:.0f}s/epoch - {timer.avg():.0f}ms/step")
        scheduler.step()
        train_loss_list.append(train_loss)
        train_acc_list.append(train_acc)
        valid_acc_list.append(valid_acc)
    measures = (f'train loss {metric[0] / metric[2]:.3f}',f'train acc {metric[1] / metric[2]:.3f}')
    # if valid_iter is not None:
    #     measures += f', valid acc {valid_acc:.3f}'
    print(f'Time {timer.sum():.1f} sec')
    # print(measures + f'\n{metric[2] * num_epochs / timer.sum():.1f}'
    #       f' examples/sec on {str(devices)}')
    # 保存训练图表
    # save_dir = 'training_plots'
    # os.makedirs(save_dir, exist_ok=True)
    # plot_filename = f"{save_dir}/train_plot_{noise_type}_{noise_level}.png"
    # plt.savefig(plot_filename)
    # plt.close()

    # 绘制训练图表
    plt.figure(figsize=(12, 4))
    plt.subplot(1, 2, 1)
    plt.plot(range(1, num_epochs + 1), train_loss_list, label='train_loss')
    plt.xlabel('Epoch')
    plt.ylabel('Loss')
    plt.title('Training Loss')
    plt.legend()

    plt.subplot(1, 2, 2)
    plt.plot(range(1, num_epochs + 1), train_acc_list, label='train_acc')
    plt.plot(range(1, num_epochs + 1), valid_acc_list, label='valid_acc')
    plt.xlabel('Epoch')
    plt.ylabel('Accuracy')
    plt.title('Training and Validation Accuracy')
    plt.legend()

    # 保存训练图表
    current_time = datetime.datetime.now().strftime('%Y%m%d_%H%M%S')
    save_dir = f'training_plots'
    os.makedirs(save_dir, exist_ok=True)
    plot_filename = f"{save_dir}/{current_time}_train_plot_{noise_type}_{noise_level}.png"
    plt.savefig(plot_filename)
    plt.close()

# devices, num_epochs, lr, wd = d2l.try_all_gpus(), 100, 2e-4, 5e-4
# lr_period, lr_decay, net = 4, 0.9, get_net()

# devices, num_epochs, lr, wd = d2l.try_all_gpus(), 100, 0.05, 5e-4
# lr_period, lr_decay, net = 50, 0.1, get_net()

# devices, num_epochs, lr, wd = d2l.try_all_gpus(), 50, 2e-4, 5e-4
# lr_period, lr_decay, net = 4, 0.9, get_net()
# # devices, num_epochs, lr, wd = d2l.try_all_gpus(), 100, 0.05, 5e-4
# # lr_period, lr_decay, net = 50, 0.1, get_net()
# train(net, train_iter, valid_iter, num_epochs, lr, wd, devices, lr_period,
#       lr_decay)


Training on NVIDIA GeForce GTX 1660 Ti
Files already downloaded and verified
Files already downloaded and verified


In [4]:
import torch
import numpy as np
import random

def add_noise_to_labels(labels, noise_ratio, noise_type='symmetric', num_classes=10):
    if noise_type not in ['symmetric', 'asymmetric']:
        raise ValueError("noise_type should be 'symmetric' or 'asymmetric'")

    noisy_labels = labels.clone()
    n = len(labels)

    # 对称噪声
    if noise_type == 'symmetric':
        for i in range(n):
            if random.random() < noise_ratio:
                noisy_labels[i] = random.choice([l for l in range(num_classes) if l != labels[i]])

    # 非对称噪声
    elif noise_type == 'asymmetric':
        print("asy: ", n, " ", noise_ratio)
        # CIFAR-10 specific label mapping
        mapping = {9: 1, 2: 0, 4: 7, 3: 5, 5: 3}
        for i in range(n):
            if random.random() < noise_ratio and labels[i].item() in mapping:
                # print("change")
                noisy_labels[i] = mapping[labels[i].item()]

    return noisy_labels
    
def verify_noise(original_labels, noisy_labels):
    assert len(original_labels) == len(noisy_labels), "Length of original and noisy labels must be equal"
    changed = (original_labels != noisy_labels).sum().item()
    total = len(original_labels)
    noise_ratio = changed / total
    return noise_ratio

# 假设 labels 是一个 PyTorch 张量，包含 CIFAR 数据集的标签

# 提取训练集的标签
original_labels = torch.tensor([full_train_ds[i][1] for i in range(len(full_train_ds))])

# 添加噪声
noise_ratio = 0.6  # 例如，添加 30% 的噪声
noisy_labels = add_noise_to_labels(original_labels, noise_ratio, noise_type='symmetric')

# 验证噪声添加
actual_noise_ratio = verify_noise(original_labels, noisy_labels)
print(f"Actual noise ratio: {actual_noise_ratio}")
print("Some original labels:", original_labels[:10])
print("Corresponding noisy labels (symmetric):", noisy_labels[:10])
# 假设 original_labels 和 noisy_labels 分别是原始和带噪声的标签
# 定义非对称噪声的映射关系
asymmetric_mapping = {9: 1, 2: 0, 4: 7, 3: 5, 5: 3}

def calculate_asymmetric_noise_ratio(original_labels, noisy_labels, mapping):
    """计算实际的非对称噪声比例"""
    n = len(original_labels)
    actual_changes = 0
    potential_changes = 0

    for i in range(n):
        if original_labels[i].item() in mapping:
            potential_changes += 1
            if noisy_labels[i].item() == mapping[original_labels[i].item()]:
                actual_changes += 1

    if potential_changes == 0:
        return 0
    else:
        return actual_changes / potential_changes


noisy_labels_a = add_noise_to_labels(original_labels, noise_ratio, noise_type='asymmetric')
# 检查原始标签和噪声标签是否有差异
print("Some original labels:", original_labels[:10])
print("Corresponding noisy labels (asymmetric):", noisy_labels_a[:10])


# 计算实际的非对称噪声比例
actual_asymmetric_noise_ratio = calculate_asymmetric_noise_ratio(original_labels, noisy_labels_a, asymmetric_mapping)
print(f"Actual asymmetric noise ratio: {actual_asymmetric_noise_ratio}")



Actual noise ratio: 0.59938
Some original labels: tensor([6, 9, 9, 4, 1, 1, 2, 7, 8, 3])
Corresponding noisy labels (symmetric): tensor([6, 9, 4, 4, 5, 1, 4, 4, 1, 3])
asy:  50000   0.6
Some original labels: tensor([6, 9, 9, 4, 1, 1, 2, 7, 8, 3])
Corresponding noisy labels (asymmetric): tensor([6, 9, 1, 4, 1, 1, 0, 7, 8, 3])
Actual asymmetric noise ratio: 0.59756


In [ ]:
def run_train_harness(noise_level=0.0, noise_type='symmetric'):
    # 重新加载数据集以避免标签被多次修改
    full_train_ds = torchvision.datasets.CIFAR10(root='../data', train=True, download=True, transform=transform_train)
    original_labels = torch.tensor(full_train_ds.targets)

    # 应用噪声
    if noise_level > 0:
        noisy_labels = add_noise_to_labels(original_labels, noise_level, noise_type)
        full_train_ds.targets = noisy_labels.tolist()

    # 分割训练集和验证集
    num_train = len(full_train_ds)
    num_valid = int(num_train * valid_ratio)
    train_ds, valid_ds = random_split(full_train_ds, [num_train - num_valid, num_valid])

    # 创建 DataLoader
    train_iter = DataLoader(train_ds, batch_size=batch_size, shuffle=True, num_workers=2)
    valid_iter = DataLoader(valid_ds, batch_size=batch_size, shuffle=False, num_workers=2)

    # 训练模型
    # net = get_net()
    devices, num_epochs, lr, wd = d2l.try_all_gpus(), 15, 2e-4, 5e-4
    lr_period, lr_decay, net = 4, 0.9, get_net()

    train(net, train_iter, valid_iter, num_epochs, lr, wd, devices, lr_period, lr_decay)

# print(f"####### Training model with 0 noise level: ########")
# run_train_harness(0, 'symmetric')
# print("\n\n\n")

noise_levels = [0.1, 0.3, 0.5, 0.8, 0.9]
noise_types = ['symmetric', 'asymmetric']
import torch
import numpy as np
import random

noise_types = ['asymmetric']
seeds = range(10)  # seeds repo

for seed in seeds:
    torch.manual_seed(seed)  # 设置随机种子
    np.random.seed(seed)  # 对于numpy操作也设置随机种子
    random.seed(seed)  # 对于Python内置的random库也设置随机种子

    for noise_type in noise_types:
        for noise_level in noise_levels:
            print(f"####### Training with Seed: {seed} type: {noise_type} noise level: {noise_level} ########")
            run_train_harness(noise_level, noise_type)
            print("\n\n\n")


####### Training with Seed: 0 type: asymmetric noise level: 0.1 ########
Files already downloaded and verified
asy:  50000   0.1


D:\miniconda3\envs\myenv\lib\site-packages\torch\nn\modules\lazy.py:180: UserWarning: Lazy modules are a new feature under heavy development so changes to the API or functionality can happen at any moment.
  warnings.warn('Lazy modules are a new feature under heavy development '


1/15 - 46s - loss: 1.5683 - accuracy: 0.4278 - val_accuracy: 0.5126 - 46s/epoch - 0ms/step
2/15 - 41s - loss: 1.0999 - accuracy: 0.6046 - val_accuracy: 0.5940 - 41s/epoch - 0ms/step
3/15 - 42s - loss: 0.8598 - accuracy: 0.6931 - val_accuracy: 0.6882 - 42s/epoch - 0ms/step
4/15 - 41s - loss: 0.7393 - accuracy: 0.7380 - val_accuracy: 0.6904 - 41s/epoch - 0ms/step
5/15 - 41s - loss: 0.6515 - accuracy: 0.7715 - val_accuracy: 0.7240 - 41s/epoch - 0ms/step
6/15 - 42s - loss: 0.5977 - accuracy: 0.7889 - val_accuracy: 0.7556 - 42s/epoch - 0ms/step
7/15 - 42s - loss: 0.5522 - accuracy: 0.8068 - val_accuracy: 0.7814 - 42s/epoch - 0ms/step
8/15 - 42s - loss: 0.5129 - accuracy: 0.8195 - val_accuracy: 0.7796 - 42s/epoch - 0ms/step
9/15 - 42s - loss: 0.4730 - accuracy: 0.8347 - val_accuracy: 0.7710 - 42s/epoch - 0ms/step
10/15 - 42s - loss: 0.4479 - accuracy: 0.8428 - val_accuracy: 0.7934 - 42s/epoch - 0ms/step
11/15 - 43s - loss: 0.4196 - accuracy: 0.8535 - val_accuracy: 0.8092 - 43s/epoch - 0ms/st

In [37]:
# noise_types = ['symmetric', 'asymmetric']
# for noise_type in noise_types:
#     noise_level = 0.1
#     print(f"####### Training with {noise_type} noise level: {noise_level} ########")
#     run_train_harness(noise_level, noise_type)
#     print("\n\n\n")

In [38]:
noise_types = ['symmetric', 'asymmetric']
for noise_type in noise_types:
    noise_level = 0.3
    print(f"####### Training with {noise_type} noise level: {noise_level} ########")
    run_train_harness(noise_level, noise_type)
    print("\n\n\n")

####### Training with symmetric noise level: 0.3 ########
Files already downloaded and verified
1/30 - 42s - loss: 2.0691 - accuracy: 0.2848 - val_accuracy: 0.3366 - 42s/epoch - 0ms/step
2/30 - 43s - loss: 1.8631 - accuracy: 0.3964 - val_accuracy: 0.4116 - 43s/epoch - 0ms/step
3/30 - 43s - loss: 1.7437 - accuracy: 0.4654 - val_accuracy: 0.4612 - 43s/epoch - 0ms/step
4/30 - 43s - loss: 1.6673 - accuracy: 0.5051 - val_accuracy: 0.4896 - 43s/epoch - 0ms/step
5/30 - 41s - loss: 1.6137 - accuracy: 0.5296 - val_accuracy: 0.5144 - 41s/epoch - 0ms/step
6/30 - 41s - loss: 1.5717 - accuracy: 0.5527 - val_accuracy: 0.5336 - 41s/epoch - 0ms/step
7/30 - 41s - loss: 1.5450 - accuracy: 0.5622 - val_accuracy: 0.5394 - 41s/epoch - 0ms/step
8/30 - 41s - loss: 1.5194 - accuracy: 0.5715 - val_accuracy: 0.5454 - 41s/epoch - 0ms/step
9/30 - 41s - loss: 1.4842 - accuracy: 0.5865 - val_accuracy: 0.5398 - 41s/epoch - 0ms/step
10/30 - 41s - loss: 1.4659 - accuracy: 0.5913 - val_accuracy: 0.5502 - 41s/epoch - 0m

In [39]:
noise_types = ['symmetric', 'asymmetric']
for noise_type in noise_types:
    noise_level = 0.5
    print(f"####### Training with {noise_type} noise level: {noise_level} ########")
    run_train_harness(noise_level, noise_type)
    print("\n\n\n")

####### Training with symmetric noise level: 0.5 ########
Files already downloaded and verified
1/30 - 41s - loss: 2.2428 - accuracy: 0.1877 - val_accuracy: 0.2134 - 41s/epoch - 0ms/step
2/30 - 42s - loss: 2.1598 - accuracy: 0.2446 - val_accuracy: 0.2618 - 42s/epoch - 0ms/step
3/30 - 41s - loss: 2.1235 - accuracy: 0.2768 - val_accuracy: 0.2748 - 41s/epoch - 0ms/step
4/30 - 41s - loss: 2.0892 - accuracy: 0.3017 - val_accuracy: 0.3018 - 41s/epoch - 0ms/step
5/30 - 41s - loss: 2.0680 - accuracy: 0.3170 - val_accuracy: 0.3172 - 41s/epoch - 0ms/step
6/30 - 41s - loss: 2.0429 - accuracy: 0.3342 - val_accuracy: 0.3304 - 41s/epoch - 0ms/step
7/30 - 41s - loss: 2.0175 - accuracy: 0.3498 - val_accuracy: 0.3334 - 41s/epoch - 0ms/step
8/30 - 41s - loss: 2.0012 - accuracy: 0.3630 - val_accuracy: 0.3384 - 41s/epoch - 0ms/step
9/30 - 41s - loss: 1.9790 - accuracy: 0.3741 - val_accuracy: 0.3520 - 41s/epoch - 0ms/step
10/30 - 41s - loss: 1.9643 - accuracy: 0.3833 - val_accuracy: 0.3620 - 41s/epoch - 0m

In [41]:
noise_types = ['symmetric', 'asymmetric']
for noise_type in noise_types:
    noise_level = 0.1
    print(f"####### Training with {noise_type} noise level: {noise_level} ########")
    run_train_harness(noise_level, noise_type)
    print("\n\n\n")

noise_types = ['symmetric', 'asymmetric']
for noise_type in noise_types:
    noise_level = 0.8
    print(f"####### Training with {noise_type} noise level: {noise_level} ########")
    run_train_harness(noise_level, noise_type)
    print("\n\n\n")

####### Training with symmetric noise level: 0.1 ########
Files already downloaded and verified
1/30 - 41s - loss: 1.7508 - accuracy: 0.3888 - val_accuracy: 0.4812 - 41s/epoch - 0ms/step
2/30 - 41s - loss: 1.3930 - accuracy: 0.5528 - val_accuracy: 0.5308 - 41s/epoch - 0ms/step
3/30 - 41s - loss: 1.1979 - accuracy: 0.6390 - val_accuracy: 0.6510 - 41s/epoch - 0ms/step
4/30 - 41s - loss: 1.0904 - accuracy: 0.6836 - val_accuracy: 0.6630 - 41s/epoch - 0ms/step
5/30 - 41s - loss: 1.0061 - accuracy: 0.7172 - val_accuracy: 0.7148 - 41s/epoch - 0ms/step
6/30 - 41s - loss: 0.9516 - accuracy: 0.7364 - val_accuracy: 0.7290 - 41s/epoch - 0ms/step
7/30 - 41s - loss: 0.9133 - accuracy: 0.7511 - val_accuracy: 0.7384 - 41s/epoch - 0ms/step
8/30 - 41s - loss: 0.8791 - accuracy: 0.7622 - val_accuracy: 0.7344 - 41s/epoch - 0ms/step
9/30 - 41s - loss: 0.8380 - accuracy: 0.7765 - val_accuracy: 0.7502 - 41s/epoch - 0ms/step
10/30 - 41s - loss: 0.8050 - accuracy: 0.7891 - val_accuracy: 0.7424 - 41s/epoch - 0m

In [1]:
noise_types = ['asymmetric']
for noise_type in noise_types:
    noise_level = 0.0
    print(f"####### Training with {noise_type} noise level: {noise_level} ########")
    run_train_harness(noise_level, noise_type)
    print("\n\n\n")

####### Training with asymmetric noise level: 0.0 ########


NameError: name 'run_train_harness' is not defined

In [43]:
noise_types = ['asymmetric']
for noise_type in noise_types:
    noise_level = 0.8
    print(f"####### Training with {noise_type} noise level: {noise_level} ########")
    run_train_harness(noise_level, noise_type)
    print("\n\n\n")

####### Training with asymmetric noise level: 0.8 ########
Files already downloaded and verified
asy:  50000   0.8
1/30 - 41s - loss: 1.4654 - accuracy: 0.4774 - val_accuracy: 0.5218 - 41s/epoch - 0ms/step
2/30 - 41s - loss: 1.0695 - accuracy: 0.6198 - val_accuracy: 0.6618 - 41s/epoch - 0ms/step
3/30 - 42s - loss: 0.8784 - accuracy: 0.6862 - val_accuracy: 0.6946 - 42s/epoch - 0ms/step
4/30 - 42s - loss: 0.7743 - accuracy: 0.7229 - val_accuracy: 0.7142 - 42s/epoch - 0ms/step
5/30 - 41s - loss: 0.6985 - accuracy: 0.7476 - val_accuracy: 0.7250 - 41s/epoch - 0ms/step
6/30 - 41s - loss: 0.6436 - accuracy: 0.7649 - val_accuracy: 0.7438 - 41s/epoch - 0ms/step
7/30 - 42s - loss: 0.6106 - accuracy: 0.7779 - val_accuracy: 0.7486 - 42s/epoch - 0ms/step
8/30 - 42s - loss: 0.5797 - accuracy: 0.7870 - val_accuracy: 0.7712 - 42s/epoch - 0ms/step
9/30 - 42s - loss: 0.5362 - accuracy: 0.7991 - val_accuracy: 0.7654 - 42s/epoch - 0ms/step
10/30 - 42s - loss: 0.5109 - accuracy: 0.8079 - val_accuracy: 0.77

In [7]:
noise_types = ['symmetric', 'asymmetric']
for noise_type in noise_types:
    noise_level = 0.9
    print(f"####### Training with {noise_type} noise level: {noise_level} ########")
    run_train_harness(noise_level, noise_type)
    print("\n\n\n")

####### Training with symmetric noise level: 0.9 ########
Files already downloaded and verified


d:\miniconda3\Lib\site-packages\torch\nn\modules\lazy.py:180: UserWarning: Lazy modules are a new feature under heavy development so changes to the API or functionality can happen at any moment.
  warnings.warn('Lazy modules are a new feature under heavy development '


1/10 - 44s - loss: 2.3296 - accuracy: 0.0991 - val_accuracy: 0.0926 - 44s/epoch - 0ms/step
2/10 - 41s - loss: 2.3045 - accuracy: 0.0999 - val_accuracy: 0.0998 - 41s/epoch - 0ms/step
3/10 - 41s - loss: 2.3038 - accuracy: 0.1015 - val_accuracy: 0.0986 - 41s/epoch - 0ms/step
4/10 - 41s - loss: 2.3035 - accuracy: 0.1024 - val_accuracy: 0.0912 - 41s/epoch - 0ms/step
5/10 - 41s - loss: 2.3029 - accuracy: 0.1041 - val_accuracy: 0.1106 - 41s/epoch - 0ms/step
6/10 - 41s - loss: 2.3024 - accuracy: 0.1058 - val_accuracy: 0.0994 - 41s/epoch - 0ms/step
7/10 - 41s - loss: 2.3023 - accuracy: 0.1059 - val_accuracy: 0.1036 - 41s/epoch - 0ms/step
8/10 - 41s - loss: 2.3016 - accuracy: 0.1073 - val_accuracy: 0.1024 - 41s/epoch - 0ms/step
9/10 - 41s - loss: 2.3012 - accuracy: 0.1089 - val_accuracy: 0.0962 - 41s/epoch - 0ms/step
10/10 - 41s - loss: 2.3006 - accuracy: 0.1096 - val_accuracy: 0.0980 - 41s/epoch - 0ms/step
Time 356.6 sec




####### Training with asymmetric noise level: 0.9 ########
Files alrea